In [31]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from DrawtexModel import DrawTexModel
from DrawtexDataset import DrawtexDataset

#### Hyperparameters

In [32]:
TRAIN_BATCH_SZ = 128
TEST_BATCH_SZ = 1000
EPOCHS = 1
LEARN_RATE = 0.01
CLASS_CNT = 82

device: str = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


#### Dataloader setup

In [33]:
data_set = DrawtexDataset(transforms.ToTensor())
TRAIN_SIZE = int(0.8 * len(data_set))
TEST_SIZE = len(data_set) - TRAIN_SIZE
train_set, test_set = torch.utils.data.random_split(data_set, [TRAIN_SIZE, TEST_SIZE])

train_loader = DataLoader(
    dataset=train_set,
    batch_size=TRAIN_BATCH_SZ,
    shuffle=True,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=TEST_BATCH_SZ,
    shuffle=False,
    num_workers=4
)

model = DrawTexModel().to(device)
print(model)

DrawTexModel(
  (relu): ReLU()
  (conv1): Conv2d(1, 200, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv1_bn): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(200, 300, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv2_bn): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(300, 500, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv3_bn): BatchNorm2d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(500, 800, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv4_bn): BatchNorm2d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(800, 1000, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv5_bn): BatchNorm2d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin1): Linear(in_features=25000, out_features=82, bias=False)
  (lin1_bn): BatchNorm1d(82, eps=1e-05, momentum=0.1, 

In [34]:
tens = torch.rand((2, 1, 45, 45)).to(device)

output1 = model(tens)

print(output1)

tensor([[ 0.9931, -0.9997, -0.9997, -0.9999,  0.9846,  1.0000, -1.0000,  1.0000,
          0.9996, -0.9996, -0.2047,  0.7502, -0.9998,  0.9999, -0.9878,  0.9999,
         -0.9917, -0.9990,  0.9977,  0.9991, -0.9999, -0.9999, -1.0000,  0.9997,
         -0.9999, -0.9999,  0.9740,  0.9997, -0.9987, -0.9999, -0.9985, -0.9996,
         -0.9999, -0.9998, -0.9998,  1.0000, -0.9979,  0.7682, -1.0000,  0.9967,
          1.0000, -0.9999,  0.9988, -0.9999, -0.9986,  0.9999, -0.9999, -0.9998,
         -1.0000, -0.9998, -0.9999, -1.0000, -0.9999,  1.0000,  0.9997, -0.9982,
          0.9999,  0.9985, -0.9994, -0.9956, -0.9975, -0.9999, -1.0000, -0.9765,
         -0.9997,  0.9999, -0.9954,  0.9994, -0.9989,  0.9999, -0.9999,  1.0000,
          1.0000, -0.9986,  0.9998,  1.0000,  1.0000, -0.9995,  0.9996,  1.0000,
          0.9999, -0.9999],
        [-0.9931,  0.9997,  0.9997,  0.9999, -0.9846, -1.0000,  1.0000, -1.0000,
         -0.9996,  0.9996,  0.2047, -0.7502,  0.9998, -0.9999,  0.9878, -0.9999,


In [35]:
def train():
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)
    steps = len(train_loader)
    for epoch in range(EPOCHS):
        for i, (img, label) in enumerate(train_loader):
            img: torch.Tensor = img.to(device, non_blocking=True)
            label: torch.Tensor = label.to(device, non_blocking=True)
            output = model(img).to(device)
            loss: torch.Tensor = criterion(output, label).to(device)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i) % 10 == 0:
                print(f"Epoch {epoch + 1}/{EPOCHS}, Batch {i + 1}/{steps}, Loss {loss.item():.4f}")

train()
torch.save(model.state_dict(), "./DrawTexModel.pth")


Epoch 1/1, Batch 1/2350, Loss 4.4067
Epoch 1/1, Batch 11/2350, Loss 4.4054
Epoch 1/1, Batch 21/2350, Loss 2.8708
Epoch 1/1, Batch 31/2350, Loss 2.6320
Epoch 1/1, Batch 41/2350, Loss 2.1396
Epoch 1/1, Batch 51/2350, Loss 2.0791
Epoch 1/1, Batch 61/2350, Loss 1.9660
Epoch 1/1, Batch 71/2350, Loss 1.7992
Epoch 1/1, Batch 81/2350, Loss 1.8196
Epoch 1/1, Batch 91/2350, Loss 1.8396
Epoch 1/1, Batch 101/2350, Loss 1.6047


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for img, labels in test_loader:
        img = img.to(device, non_blocking= True)
        labels = labels.to(device, non_blocking= True)

        output = model(img)
        _, prediction = torch.max(output, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()

    acc = 100.0 * correct / total
    print(f"Accuracy: {acc}%")